### ITCR - Escuela de Computación
### Curso IC-6200 - Inteligencia Artificial
### Aprendizaje supervisado

### Redes de memoria de corto y largo plazo con PyTorch 
### (Long-Short Term Memory Networks-LSTM)

**Profesora: María Auxiliadora Mora**

### Tarea corta #7

### Estudiantes:
**1. Pablo Alberto Muñoz Hidalgo**

**2. Luis Andrés Rojas Murillo**


## Introducción

La clasificación de textos y el reconocimiento de entidades nombradas (Named Entity Recognition o NER por sus siglas en inglés) son técnicas fundamentales que constituyen el primer paso en muchas tareas de Procesamiento de lenguaje natural (NLP). NER, es un área de investigación relacionada a la extracción de información, que permite localizar y clasificar nombres de entidades que se encuentran en texto libre, en categorías comunmente organizaciones, lugares, tiempo, personas, entre otros. Ejemplo:

- El fundador de [Microsoft Corporation] (organización), [Bill Gates] (persona), comentó que se abrirán 1000 puestos de trabajo en la [Región Chorotega] (lugar) a partir del año 2022 (fecha).  

La clasificación de textos permite categorizar el contenido asociando este a un conjunto de etiquetas predefinidas o clases. Su uso más popular es el análisis de sentimientos. Ejemplo:

- En mi opinión, la película fue muy buena porque pudo dar a conocer a los espectadores cómo puede afectar una situación traumática a la mente humana. (Clase = 5 o excelente). 

Las redes neuronales recurrentes o RNN (Rumelhart et al., 1986, como se citó en LeCun et al., 2015) son una familia de redes neuronales para el procesamiento de secuencias de datos, las cuales en un tiempo t, reciben el estado anterior, es decir, su salida en el tiempo t podría usarse como insumo del procesamiento de la siguiente entrada, de modo que la información pueda propagarse a medida que la red pasa por la secuencia de entrada. Las redes Long Short-Term Memory (LSTM) son un tipo de red neuronal recurrente capaz de aprender dependencias a largo plazo.

El siguiente ejemplo implementa NER con una LSTM para etiquetar el rol que juegan las palabras en las oraciones. 


## Ejemplo

El sistema implementado en el código adjunto soluciona el problema de estimar el rol de una palabra en una frase, por ejempo roles como determinante (DET), nombre (NN) y verbo (V). 
Ejemplo para la frase:

- "El perro come manzana" la salida deberá ser: ["DET", "NN", "V", "NN"]). 

Este proceso se conoce en el procesamiento de lenguaje natural como "part of speech tagging (POS)".

Este es un ejemplo simple con datos introducidos en el código basado en [1].

Se realizarán los siguientes pasos

   * Definición de los ejemplos (codificados) 
   * Preprocesamiento de las palabras a clasificar
   * Definición del modelo
   * Instanciación del modelo, definición de la función de pérdida y del optimizador  
   * Entrenamiento de la red
   * Pruebas del modelo resultante con unos cuantos ejemplos.


In [29]:
# Bibliotecas requeridas

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import csv
import random
import numpy as np
#Import scikit learn metrics
from sklearn import metrics



torch.manual_seed(1)

In [30]:
# Funciones utilitarias

def max_values(x):
    """
    Retorna el valor máximo y en índice o la posición del valor en un vector x.
    Parámetros: 
        x: vector con los datos. 
    Salida: 
        out: valor 
        inds: índice
    """
    out, inds = torch.max(x,dim=1)   
    return out, inds
    

    # Preparación de los datos 
def prepare_sequence(seq, to_ix):
    """
    Retorna un tensor con los indices del diccionario para cada palabras en una oración.
    Parámetros:
       seq: oración
       to_ix: diccionario de palabras.
    """
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


In [31]:
#limpieza de datos 
# Read the data from the CSV file
labels = ["DET", "NN", "V", "ADJ", "PREP"]
with open('FULL DATA.csv', 'r') as file:
    reader = csv.reader(file)
    data = list(reader)

with open('FULL DATA.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerows(data)
counter = 0
list_to_delete = []

#todas las frases con otra dificultad
for i in data:
    if len(i) > 8 or len(i) < 8:
        counter += 1
        list_to_delete.append(i)

for i in list_to_delete:
    data.remove(i)
list_to_delete = []

#todas las frases con diferete logitud
for i in data:
    sentence = i[0].split()
    if len(sentence) > 7 or len(sentence) < 7:
        counter += 1
        list_to_delete.append(i)

for i in list_to_delete:
    data.remove(i)
list_to_delete = []

#todas las frases con etiquetas no reconocidas
for i in data:
    for j in range(len(i)):
        if j > 0 and i[j] not in labels:
            counter += 1
            list_to_delete.append(i)


for i in list_to_delete:
    if i in data:
        data.remove(i)
list_to_delete = []

print("Numero de frases con errores: ", counter)
print("Numero de frases sin errores: ", len(data))


Numero de frases con errores:  5147
Numero de frases sin errores:  71


In [32]:


# Divide the data into training and testing sets
#random.shuffle(data)
split = int(0.8 * len(data))
training_data = data[:split]
test_data = data[split:]


# Split the training data into text and labels
for i in range(len(training_data)):
    splitted_text = training_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(training_data[i])):
        if j != 0:
            labels.append(training_data[i][j])
    training_data[i] = [0,0]
    training_data[i][0] = splitted_text
    training_data[i][1] = labels

# Split the test data into text and labels
for i in range(len(test_data)):
    splitted_text = test_data[i][0].split()
    #put the rest of i in other list
    labels = []
    for j in range(len(test_data[i])):
        if j != 0:
            labels.append(test_data[i][j])
    test_data[i] = [0,0]
    test_data[i][0] = splitted_text
    test_data[i][1] = labels

# print the data
print("Training data:")
print(training_data)
print("Test data:")
print(test_data)


# Diccionario las palabras
word_to_ix = {}
for sent, tags in training_data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print("Diccionario", word_to_ix)

# Asignar índices a las etiquetas
tag_to_ix = {"DET": 0, "NN": 1, "V": 2, "ADJ":3, "PREP":4}

#print the amount of each tag found in the text, print that in a table using pandas

print("\n\n\n\nCantidad de etiquetas encontradas en el texto:")
for tag in tag_to_ix:
    count = 0
    for sent, tags in training_data + test_data:
        for word in tags:
            if word == tag:
                count += 1
    print(tag," : ", count)


for i in range(len(training_data)):
    #put the rest of i in other list
    if len(training_data[i][1]) == 6:
        print(training_data[i])

for i in range(len(test_data)):
    #put the rest of i in other list
    if len(test_data[i][1]) == 6:
        print(test_data[i])



Training data:
[[['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardin'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'mesa', 'grande', 'esta', 'en', 'la', 'cocina'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'perro', 'marron', 'ladra', 'en', 'el', 'parque'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'libro', 'interesante', 'esta', 'en', 'la', 'biblioteca'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'ninia', 'pequenia', 'juega', 'en', 'el', 'patio'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'pelota', 'roja', 'rueda', 'en', 'el', 'suelo'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'hombre', 'mayor', 'camina', 'por', 'la', 'calle'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['La', 'mujer', 'guapa', 'esta', 'en', 'el', 'parque'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'ninio', 'feliz', 'juega', 'en', 'la', 'playa'], ['DET', 'NN', 'ADJ', 'V', 'PREP', 'DET', 'NN']], [['El', 'coc

In [33]:
# Ejemplo de procesamiento de una oración
inputs = prepare_sequence(training_data[0][0], word_to_ix)
print(training_data[0][0])                          
print(inputs)

['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardin']
tensor([0, 1, 2, 3, 4, 5, 6])


In [34]:
# Definición del modelo

# El modelo es una clase que debe heredar de nn.Module
class LSTMTagger(nn.Module):
    """
    Clase para aplicar POST a oraciones en español. 
    """
    
    # Incialización del modelo
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        """
        Inicialización de la clase.
        Parámetros:
           embedding_dim: dimesionalidad del vector de palabras. 
           hidden_dim: dimensión de la capa oculta de la red. 
           vocab_size: tamaño del vocabulario.  
           tagset_size: número de clases.
        """
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # Primero se pasa la entrada a través de una capa Embedding. 
        # Esta capa construye una representación de los tokens de 
        # un texto donde las palabras que tienen el mismo significado 
        # tienen una representación similar.
        
        # Esta capa captura mejor el contexto y son espacialmente 
        # más eficientes que las representaciones vectoriales (one-hot vector).
        # En Pytorch, se usa el módulo nn.Embedding para crear esta capa, 
        # que toma el tamaño del vocabulario y la longitud deseada del vector 
        # de palabras como entrada. Ejemplos en [3] y [4]
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # El LSTM toma word_embeddings como entrada y genera los estados ocultos
        # con dimensionalidad hidden_dim.  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # La capa lineal mapea el espacio de estado oculto 
        # al espacio de clases
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        # Pasada hacia adelante de la red. 
        # Parámetros:
        #    sentence: la oración a procesar
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))

        # Se utiliza softmax para devolver la probabilidad de cada etiqueta
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    



In [35]:

# Instanciación del modelo, definición de la función de pérdida y del optimizador   

# Hiperparámetros de la red
# Valores generalmente altos (32 o 64 dimensiones).
#Nodos se subieron a 10 con la intencion para experimentar
EMBEDDING_DIM = 10
HIDDEN_DIM = 10



# Instancia del modelo
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))

# Función de pérdida: Negative Log Likelihood Loss (NLLL). 
# Generalmente utilizada en problemas de clasificacion con múltiples clases.
loss_function = nn.NLLLoss()

# Optimizador Stochastic Gradient Descent  
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [36]:
# Entrenamiento del modelo 

# Valores antes de entrenar
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    
    print(training_data[0][0])
    
    # Clasificación    
    print(tag_scores)

acc =0
# Épocas de entrenamiento
for epoch in range(200):  
    for sentence, tags in training_data:
        ## Paso 1. Pytorch acumula los gradientes.
        # Es necesario limpiarlos
        model.zero_grad()

        # Paso 2. Se preparan las entradas, es decir, se convierten a
        # tensores de índices de palabras.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Paso 3. Se genera la predicción (forward pass).
        tag_scores = model(sentence_in)

        # Paso 4. se calcula la pérdida, los gradientes, y se actualizan los 
        # parámetros por medio del optimizador.
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
    

# Despligue de la puntuación luego del entrenamiento
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
   
    print("Resultados luego del entrenamiento para la primera frase")
    print(tag_scores)

['El', 'gato', 'negro', 'corre', 'en', 'el', 'jardin']
tensor([[-1.6499, -1.6724, -1.4370, -1.8541, -1.4876],
        [-1.4980, -1.7850, -1.4924, -2.0177, -1.3830],
        [-1.4917, -1.8383, -1.4603, -1.9947, -1.3955],
        [-1.4995, -1.7520, -1.5521, -1.9660, -1.3803],
        [-1.5165, -1.7765, -1.5060, -2.0250, -1.3566],
        [-1.5668, -1.7597, -1.4809, -2.0213, -1.3499],
        [-1.5467, -1.8426, -1.4104, -1.9674, -1.4073]])
Resultados luego del entrenamiento para la primera frase
tensor([[-3.1145e-04, -8.2100e+00, -1.4919e+01, -1.2563e+01, -1.0242e+01],
        [-8.1076e+00, -1.3618e-03, -1.3716e+01, -6.8509e+00, -1.5676e+01],
        [-1.2281e+01, -8.1560e+00, -6.9182e+00, -1.2858e-03, -1.2522e+01],
        [-1.3795e+01, -1.2786e+01, -1.9536e-03, -6.4533e+00, -7.8954e+00],
        [-9.7351e+00, -1.5004e+01, -7.7160e+00, -1.1209e+01, -5.1878e-04],
        [-8.5758e-04, -7.6208e+00, -1.4294e+01, -1.3064e+01, -7.9175e+00],
        [-8.0319e+00, -4.5718e-04, -1.5568e+01, -8.9

#### Precision score

In [37]:
def get_precision_score(model, data):
    correct = 0
    total = 0
    for sentence, tags in data:
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        for i in range(len(tag_scores)):
            total += 1
            if torch.argmax(tag_scores[i]) == targets[i]:
                correct += 1
    return correct/total

print("Precision score for training data: ", get_precision_score(model, training_data)*100, "%")



Precision score for training data:  100.0 %


#### Exhaustividad

In [38]:
def recall_score(y_test, y_pred, average):
    if average == "macro":
        recall = 0
        for i in range(len(y_test)):
            if y_test[i] == y_pred[i]:
                recall += 1
        return recall/len(y_test)
    elif average == "micro":
        recall = 0
        for i in range(len(y_test)):
            if y_test[i] == y_pred[i]:
                recall += 1
        return recall/len(y_test)
    elif average == "weighted":
        recall = 0
        for i in range(len(y_test)):
            if y_test[i] == y_pred[i]:
                recall += 1
        return recall/len(y_test)
    else:
        print("Invalid average parameter")

#### F1

In [39]:
def f1_score(y_true, y_pred, average='macro'):
    recall = recall_score(y_test, y_pred, average='macro')
    precision = get_precision_score(model, training_data)
    return 2 * (precision * recall) / (precision + recall)

y_pred = []
for sentence, tags in test_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = prepare_sequence(tags, tag_to_ix)
    tag_scores = model(sentence_in)
    for i in range(len(tag_scores)):
        y_pred.append(torch.argmax(tag_scores[i]))


y_test = []
for sentence, tags in test_data:
    sentence_in = prepare_sequence(sentence, word_to_ix)
    targets = prepare_sequence(tags, tag_to_ix)
    for i in range(len(targets)):
        y_test.append(targets[i])
        
f1_score = f1_score(y_test, y_pred, average='macro')
print("F1 score: ", f1_score*100, "%")


F1 score:  97.05882352941177 %


In [40]:
# Uso del modelo generado
def test_examples(test_data):

   with torch.no_grad():
      inputs = prepare_sequence(test_data, word_to_ix)
      tag_scores = model(inputs)
    
 
   print("FRASE") 
   print("La frase original", test_data)    
   print("La frase original preprocesada", inputs)
   print("Salida del modelo", tag_scores)
   print("Valores máximos e índices", max_values(tag_scores))    
    
print("Clases")
print(tag_to_ix)

#Frase 1
test_examples(test_data[0][0])


Clases
{'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}
FRASE
La frase original ['La', 'maleta', 'pesada', 'es', 'dificil', 'de', 'llevar']
La frase original preprocesada tensor([  7, 146, 147, 142, 148, 149, 150])
Salida del modelo tensor([[-1.3838e-03, -6.6321e+00, -1.1651e+01, -1.0529e+01, -1.0415e+01],
        [-7.2527e+00, -1.9346e-03, -1.2886e+01, -6.7082e+00, -1.3741e+01],
        [-1.2640e+01, -6.2074e+00, -5.6051e+00, -5.7328e-03, -1.0837e+01],
        [-1.0337e+01, -1.0571e+01, -3.2290e-02, -4.9976e+00, -3.6904e+00],
        [-5.6293e+00, -8.9339e+00, -2.5343e+00, -4.8255e+00, -9.5480e-02],
        [-1.2010e-01, -3.1737e+00, -9.4935e+00, -9.1159e+00, -2.6432e+00],
        [-1.1699e+01, -4.2418e-04, -1.2586e+01, -7.7982e+00, -1.3259e+01]])
Valores máximos e índices (tensor([-0.0014, -0.0019, -0.0057, -0.0323, -0.0955, -0.1201, -0.0004]), tensor([0, 1, 3, 2, 4, 0, 1]))


In [41]:
#Frase 2
test_examples(test_data[3][0])

print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['El', 'cuaderno', 'nuevo', 'esta', 'en', 'la', 'mochila']
La frase original preprocesada tensor([  0, 107, 161,  10,   4,  11, 162])
Salida del modelo tensor([[-3.1145e-04, -8.2100e+00, -1.4919e+01, -1.2563e+01, -1.0242e+01],
        [-9.7018e+00, -3.7472e-04, -1.3896e+01, -8.0716e+00, -1.5072e+01],
        [-1.1576e+01, -6.6076e+00, -6.9404e+00, -2.3400e-03, -1.1531e+01],
        [-1.4923e+01, -1.3253e+01, -1.1129e-03, -7.1394e+00, -8.0569e+00],
        [-9.8896e+00, -1.5076e+01, -7.7739e+00, -1.1367e+01, -4.8328e-04],
        [-6.8558e-04, -7.7612e+00, -1.4287e+01, -1.3421e+01, -8.2650e+00],
        [-6.4974e+00, -2.2295e-03, -1.2840e+01, -7.2442e+00, -1.2805e+01]])
Valores máximos e índices (tensor([-0.0003, -0.0004, -0.0023, -0.0011, -0.0005, -0.0007, -0.0022]), tensor([0, 1, 3, 2, 4, 0, 1]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}


In [42]:
# Otra prueba
test_examples(test_data[2][0])
print("valor de las etiquetas", tag_to_ix)

FRASE
La frase original ['La', 'puerta', 'cerrada', 'no', 'permite', 'el', 'paso']
La frase original preprocesada tensor([  7, 156, 157, 158, 159,   5, 160])
Salida del modelo tensor([[-1.3838e-03, -6.6321e+00, -1.1651e+01, -1.0529e+01, -1.0415e+01],
        [-6.9696e+00, -3.6249e-03, -1.2117e+01, -5.9252e+00, -1.3306e+01],
        [-1.2493e+01, -7.0096e+00, -6.9012e+00, -1.9216e-03, -1.1991e+01],
        [-8.7841e+00, -1.1287e+01, -1.9371e-02, -4.0130e+00, -6.9710e+00],
        [-5.2524e+00, -1.1763e+01, -3.1264e+00, -6.2656e+00, -5.2367e-02],
        [-1.5425e-03, -6.8375e+00, -1.3948e+01, -1.2726e+01, -7.6741e+00],
        [-8.5009e+00, -5.8479e-04, -1.3878e+01, -7.8750e+00, -1.5050e+01]])
Valores máximos e índices (tensor([-0.0014, -0.0036, -0.0019, -0.0194, -0.0524, -0.0015, -0.0006]), tensor([0, 1, 3, 2, 4, 0, 1]))
valor de las etiquetas {'DET': 0, 'NN': 1, 'V': 2, 'ADJ': 3, 'PREP': 4}


## Conclusiones

1. En el caso de LSTM consideramos que es un tipo de red neuronal recurrente especialmente eficaz para procesar datos secuenciales. EL punto fuerte de LSTM es que puede recordar información del pasado y utilizarla para realizar predicciones en el futuro. Esto hace que las LSTM sean especialmente útiles para tareas como el procesamiento del lenguaje natural, el reconocimiento del habla, el análisis de vídeo, entre otros.
2. Nos dimos cuenta que el LSTM no solo se puede aplicar al NLP, también tiene otras aplicaciones en las que por su memoria y capacidad de recordar el pasado para aplicarlo al futuro se convierte útil en aplicaciones en áreas como las finanzas, la medicina, la robótica y la composición musical. Por ejemplo, investigando un poco sobre LSTM encontramos que en finanzas, LSTM se ha utilizado para predecir los precios de las acciones, mientras que en medicina se ha empleado para el diagnóstico de enfermedades y el descubrimiento de fármacos. Por lo que LSTM no solo se limita al NLP y eso lo hace muy valioso.
3. También consideramos de vital importancia el tema del NLP el cuál no se debe minimizar ni subestimar ya que es uno subcampos que ha evolucionado rápidamente en los últimos años debido a los avances en las técnicas de aprendizaje automático, la disponibilidad de grandes cantidades de datos de texto y el desarrollo de recursos informáticos más potentes. El mismo subcampo del NLP tiene una amplia gama de aplicaciones, desde los chatbots y los asistentes virtuales hasta el análisis de sentimientos, la traducción automática y la extracción de información.
4. Nos parecio muy interesante el tema del POS y como este define mas o menos las instrucciones para la red neuronal, creemos muy importante conocer eso sin embargo nos llama mucho la atención el como podría funcionar una red neuronal no supervisada la cúal casi que no tenga parametros o formas de saber como dividir los datos, solo se le provean resultados. Sin emabrgo la precisión de estos algoritmos varía en función de la complejidad y ambigüedad de la lengua analizada, en el caso del español se tuvo que "normalizar" los datos para que no hubieran demasiados problemas.



PD: Profe, no estamos muy seguros de si funciona del todo bien el tema de precision, exhaustividad y F1 entonces por razones de tiempo lo entregamos así, sin embargo seguiremos investigando al respecto.

# Referencias 

[1] Guthrie, R. (2017). Tutorial. Sequence Models and Long-Short Term Memory Networks. Recuperado de https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

[2] LeCun,Y., Bengio, Y.,  & Hinton, G. (2015). Deep learning. Nature, 521(7553):436.

[3] Brownlee, J. (2017). What Are Word Embeddings for Text?. Recuperado de https://machinelearningmastery.com/what-are-word-embeddings/

[4] Bishop, C (2006). Pattern Recognition and Machine Learning. Springer.